# 02 - Data from the Web

## Deadline
Wednesday October 25, 2017 at 11:59PM

## Important Notes
* Make sure you push on GitHub your Notebook with all the cells already evaluated (i.e., you don't want your colleagues to generate unnecessary Web traffic during the peer review)
* Don't forget to add a textual description of your thought process, the assumptions you made, and the solution you plan to implement!
* Please write all your comments in English, and use meaningful variable names in your code.

## Background
In this homework we will extract interesting information from www.topuniversities.com and www.timeshighereducation.com, two platforms that maintain a global ranking of worldwide universities. This ranking is not offered as a downloadable dataset, so you will have to find a way to scrape the information we need!
You are not allowed to download manually the entire ranking -- rather you have to understand how the server loads it in your browser. For this task, Postman with the Interceptor extension can help you greatly. We recommend that you watch this [brief tutorial](https://www.youtube.com/watch?v=jBjXVrS8nXs&list=PLM-7VG-sgbtD8qBnGeQM5nvlpqB_ktaLZ&autoplay=1) to understand quickly how to use it.

## Assignment
* Obtain the 200 top-ranking universities in www.topuniversities.com ([ranking 2018](https://www.topuniversities.com/university-rankings/world-university-rankings/2018)). In particular, extract the following fields for each university: name, rank, country and region, number of faculty members (international and total) and number of students (international and total). Some information is not available in the main list and you have to find them in the [details page](https://www.topuniversities.com/universities/ecole-polytechnique-fÃ©dÃ©rale-de-lausanne-epfl).Store the resulting dataset in a pandas DataFrame and answer the following questions:
  * Which are the best universities in term of: (a) ratio between faculty members and students, (b) ratio of international students?

  * Answer the previous question aggregating the data by (c) country and (d) region.

  Plot your data using bar charts and describe briefly what you observed.

In [ ]:
# Import libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd 
import json 

import seaborn as sns

# set the default color palette
# try other options: 'Blues', sns.cubehelix_palette(8)
sns.set_palette('Blues')

# Seaborn can also use a context for different purpose
# possible values are paper, notebook, talk, and poster
sns.set_context("notebook")


import matplotlib.pyplot as plt
%matplotlib inline

import pickle

In [ ]:
# get name, rank, country and region of the 200 first universities
url='https://www.topuniversities.com/sites/default/files/qs-rankings-data/357051.txt?_=1508005996450'
universities= requests.get(url).json()['data'][0:200]
df=pd.DataFrame(universities)
df = df.drop(['cc','core_id','guide', 'logo','score','stars','nid'], axis=1)

#list of urls to universities pages
urls =list(df['url'])

# function to get the rest of the needed information on each university page in a dictionnary
def get_details(url):
    university= requests.get('https://www.topuniversities.com' + url)
    soup= BeautifulSoup(university.text,'html.parser')
    data= soup.find_all('div', class_= 'faculty-main')
    df.head()

    # number of faculty members international
    if(soup.find_all('div', class_ = 'inter faculty') != []):
        inter_fac_mem = float(soup.find_all('div', class_ = 'inter faculty')[0].find_all('div', class_ = 'number')[0].string.replace(',',''))
    else:
        inter_fac_mem = float('nan')
        
    # number of faculty members total
    if(soup.find_all('div', class_ = 'total faculty')!= []):
        total_fac_mem = float(soup.find_all('div', class_ = 'total faculty')[0].find_all('div', class_ = 'number')[0].string.replace(',',''))
    else:
        total_fac_mem = float('nan')
        
    # number of students international
    if(soup.find_all('div', class_ = 'total inter')!= []):
        inter_student = float(soup.find_all('div', class_ = 'total inter')[0].find_all('div', class_ = 'number')[0].string.replace(',',''))
    else:
        inter_student = float('nan')
        
    # number of students total
    if(soup.find_all('div', class_ = 'total student')!= []):
        total_student = float(soup.find_all('div', class_ = 'total student')[0].find_all('div', class_ = 'number')[0].string.replace(',',''))
    else:
        total_student = float('nan')

    details = {'url' : url, '#faculty members international (QS)' : inter_fac_mem,
          '#faculty members total (QS)' : total_fac_mem,
          '#students international (QS)': inter_student,
          '#students total (QS)' : total_student}
    return details

#list of dictionnaries 
details = []    
for url in urls:
    details.append(get_details(url))
#convert to pd.DataFrame
df2 = pd.DataFrame(details)

#merge the two dataframes with the url
df_qs= pd.merge(df, df2, 'outer')
df_qs.rename(columns={'rank_display': 'QS rank', 'title': 'name'}, inplace=True)
df_qs = df_qs.drop('url', axis=1)

In [ ]:
#USE PICKLE TO SAVE DF
filename="./data/QS_ranking"
df_qs.to_pickle(filename)

In [ ]:
#USE PICKLE TO LOAD DF
#filename="./data/QS_ranking"
#df_qs =pd.read_pickle(filename)
#df_qs.head()


In [ ]:
df_qs['ratio between faculty members and students'] = df_qs['#faculty members total (QS)']/df_qs['#students total (QS)']
df_qs['ratio of international students'] = df_qs['#students international (QS)']/df_qs['#students total (QS)']

In [ ]:
df_ratio1 = df_qs.sort_values('ratio between faculty members and students', ascending =False)
df_ratio1.set_index(['name'],inplace=True)
df_ratio1= df_ratio1['ratio between faculty members and students']

ax = df_ratio1[0:10].plot.bar()
ax.set_xlabel("Universities", fontsize=12)
plt.title("Ratio between faculty members and students")
plt.show()


In [ ]:
df_ratio2 = df_qs.sort_values('ratio of international students', ascending =False)
df_ratio2.set_index(['name'],inplace=True)
df_ratio2= df_ratio2['ratio of international students']

ax = df_ratio2[0:10].plot.bar()
ax.set_xlabel("Universities", fontsize=12)
plt.title("Ratio of international students")
plt.show()

In [ ]:
df_country = df_qs.groupby('country').mean()
df_region = df_qs.groupby('region').mean()

In [ ]:
df_country_ratio1 = df_country.sort_values('ratio between faculty members and students', ascending =False)
df_country_ratio1= df_country_ratio1['ratio between faculty members and students']

ax = df_country_ratio1[0:10].plot.bar()
ax.set_xlabel("Country", fontsize=12)
plt.title("Ratio between faculty members and students")
plt.show()

In [ ]:
df_country_ratio2 = df_country.sort_values('ratio of international students', ascending =False)
df_country_ratio2= df_country_ratio2['ratio of international students']

ax = df_country_ratio2[0:10].plot.bar()
ax.set_xlabel("Country", fontsize=12)
plt.title("Ratio of international students")
plt.show()

In [ ]:
df_region_ratio1 = df_region.sort_values('ratio between faculty members and students', ascending =False)
df_region_ratio1= df_region_ratio1['ratio between faculty members and students']

ax = df_region_ratio1.plot.bar()
ax.set_xlabel("Region", fontsize=12)
plt.title("Ratio between faculty members and students")
plt.show()

In [ ]:
df_region_ratio2 = df_region.sort_values('ratio of international students', ascending =False)
df_region_ratio2= df_region_ratio2['ratio of international students']

ax = df_region_ratio2.plot.bar()
ax.set_xlabel("Region", fontsize=12)
plt.title("Ratio of international students")
plt.show()

* Obtain the 200 top-ranking universities in www.timeshighereducation.com ([ranking 2018](http://timeshighereducation.com/world-university-rankings/2018/world-ranking)). Repeat the analysis of the previous point and discuss briefly what you observed.

* Merge the two DataFrames created in questions 1 and 2 using university names. Match universities' names as well as you can, and explain your strategy. Keep track of the original position in both rankings.

* Find useful insights in the data by performing an exploratory analysis. Can you find a strong correlation between any pair of variables in the dataset you just created? Example: when a university is strong in its international dimension, can you observe a consistency both for students and faculty members?

* Can you find the best university taking in consideration both rankings? Explain your approach.

Hints:
- Keep your Notebook clean and don't print the verbose output of the requests if this does not add useful information for the reader.
- In case of tie, use the order defined in the webpage.

In [ ]:
# Import libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd


In [ ]:

url='https://www.timeshighereducation.com/sites/default/files/the_data_rankings/world_university_rankings_2018_limit0_369a9045a203e176392b9fb8f8c1cb2a.json'

json_dict= requests.get(url).json()
raw_data = pd.DataFrame(json_dict['data'])
raw_data.columns
data=raw_data[['name','rank','location','stats_number_students','stats_pc_intl_students','stats_student_staff_ratio']]

data['rank']=data['rank'].str.extract('(\d+)').astype('int64')
data = data.loc[data['rank']<201]
data.rename(columns={'stats_number_students':'students','stats_pc_intl_students':'international students','stats_student_staff_ratio':'staff'},inplace=True)

data['students'] = data['students'].str.replace(',','').astype('int64')

data['international students'] = data['international students'].str.extract('(\d+)').astype('float64')
data['international students'] = (data['international students']/100)*data['students'].astype('float64')
data['international students'] = data['international students'].astype('int64')

data['staff'] = data['staff'].astype('float64')
data['staff'] = data['students'].astype('float64')/data['staff']
data['staff'] = data['staff'].astype('int64')

data

In [ ]:
data['staff ratio']=data['staff'].astype('float64')/data['students']
data.set_index()
barplot = data.sort_values('staff ratio',ascending=False)['staff ratio'].head(10).plot(kind='bar')

In [ ]:
data['international students ratio']=data['international students'].astype('float64')/data['students']
data.sort_values('international students ratio',ascending=False).head()

In [ ]:
data.groupby(['location'])['staff ratio'].mean().sort_values(ascending=False).plot(kind='bar')

In [ ]:
data.groupby(['location'])['international students ratio'].mean().sort_values(ascending=False).plot(kind='bar')

In [ ]:
region = {'Asia':['China','Hong Kong','Japan','Singapore','South Korea','Taiwan']
          ,'Europe':['Austria','Belgium','Denmark','Finland','France','Germany','Ireland','Italy','Luxembourg','Netherlands','Norway','Russian Federation','Spain','Sweden','Switzerland','United Kingdom']
          ,'North America':['Canada','United States']
          ,'Africa':['South Africa']
          ,'Oceania':['Australia','New Zealand']}

region = { f : i for i in region for f in region[i] }

data['region'] = data['location'].map(region)

data.groupby(['region'])['staff ratio'].mean().sort_values(ascending=False).plot(kind='bar')

In [ ]:
data.groupby(['region'])['international students ratio'].mean().sort_values(ascending=False).plot(kind='bar')

In [ ]:
data.groupby(['region'])['rank'].count().sort_values(ascending=True).plot(kind='bar')